In [8]:
def generate_complete_list_of_journals():

    import glob
    import pandas as pd

    journal_names = glob.glob("scimago*.csv")

    records = []
    for journal_name in journal_names:
        records.append(
            pd.read_csv(
                journal_name,
                sep=";",
                on_bad_lines="skip",
            )
        )
    records = pd.concat(records)
    records = records[["Title"]]
    records = records.drop_duplicates()
    records = records.sort_values(by="Title")
    records.to_csv("complete_list_of_journals.txt", index=False, header=False)


# generate_complete_list_of_journals()

In [14]:
def process_candidates():

    import glob
    import pandas as pd

    #
    # Extract existent source titles
    existent_titles = glob.glob("journals/*.csv")
    existent_titles = [title[:-4] for title in existent_titles]

    #
    # Load candidate records
    file_names = glob.glob("candidates/*.csv")

    records = []
    for file_name in file_names:
        records.append(
            pd.read_csv(
                file_name,
                sep=",",
                on_bad_lines="skip",
            )
        )
    records = pd.concat(records)

    #
    # Extract candidate source titles
    candidate_titles = records[["Source title"]].drop_duplicates()

    #
    # Remove existent titles from candidate titles
    candidate_titles = candidate_titles[
        ~candidate_titles["Source title"].isin(existent_titles)
    ]

    print(candidate_titles)

    #
    # For each candidate title, filter the records and save the dataframe to a separate CSV file
    # in the journals directory
    for candidate_title in candidate_titles["Source title"]:
        records[records["Source title"] == candidate_title].to_csv(
            f"journals/{candidate_title.replace('/', ' ')}.csv",
            index=False,
            sep=",",
        )


process_candidates()